In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy import stats

In [5]:
data = pd.read_csv("cars93.csv")

datax = data[["MPG.city", "MPG.highway", "Horsepower"]]
datax["MPG.city"] = (datax["MPG.city"] - datax["MPG.city"].min()) / (datax["MPG.city"].max() - datax["MPG.city"].min())
datax["MPG.highway"] = (datax["MPG.highway"] - datax["MPG.highway"].min()) / (datax["MPG.highway"].max() - datax["MPG.highway"].min())
datax["Horsepower"] = (datax["Horsepower"] - datax["Horsepower"].min()) / (datax["Horsepower"].max() - datax["Horsepower"].min())
datax = datax.to_numpy()
datay = data["Price"].to_numpy()
x_train, x_test, y_train, y_test = train_test_split(datax, datay, random_state=1488, test_size=0.25)

print(x_test.shape)
print(y_test.shape)

(24, 3)
(24,)


/tmp/ipykernel_4308/3156339589.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datax["MPG.city"] = (datax["MPG.city"] - datax["MPG.city"].min()) / (datax["MPG.city"].max() - datax["MPG.city"].min())
/tmp/ipykernel_4308/3156339589.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datax["MPG.highway"] = (datax["MPG.highway"] - datax["MPG.highway"].min()) / (datax["MPG.highway"].max() - datax["MPG.highway"].min())
/tmp/ipykernel_4308/3156339589.py:6: SettingWithCopyWarning: 
A value is trying to be set o

In [6]:
def predict(x, weights, b):
    return np.dot(x, weights)+b

def do_linear_regression(x_train, y_train):
    b = np.random.rand()
    weights = np.array([np.random.rand() for i in range(x_train.shape[1])])
    for _ in range(25):
        for i in range (x_train.shape[0]):
            pred = predict(x_train[i], weights, b)
            loss =(pred - y_train[i])**2
            
            dw = 2 * x_train[i] * (pred - y_train[i])
            db = 2 * (pred - y_train[i])
            
            weights -= 0.01 * dw
            b -= 0.01 * db
        print(loss)
    return b, weights

In [7]:
bias, weights = do_linear_regression(x_train, y_train)

41.1815085674396
68.15298717861681
63.085365919866
54.08117357822006
46.06306430018948
39.46030658153491
34.07520484144604
29.672903221315096
26.057426533916114
23.073081553133516
20.596953119605082
18.53183791153432
16.800606135420775
15.341821864863746
14.106353073555916
13.054744872002203
12.155179743012667
11.381889600911979
10.713916177706874
10.134140452703203
9.628520322637415
9.185489819287781
8.795483962718313
8.45056158382802
8.144104762819945


In [8]:
predict(x_test, weights, bias)

array([14.32504531, 12.09762684, 18.66382513, 17.66506814, 15.5044563 ,
       18.73390545, 14.67493591, 21.26473973,  8.67290055, 24.33503304,
       22.20887821, 23.40812307,  9.69215973, 19.35613093, 19.3541939 ,
       27.51296703, 12.60406791, 35.79954235, 22.62363841, 18.49979926,
       20.54839837, 25.69653708, 23.23317777, 15.65402938])

In [9]:
y_test

array([13.4, 10.3, 19.7, 10. , 14.1, 31.9, 11.4, 18.4,  7.4, 16.3, 17.7,
       26.3,  9.8, 20.2, 15.8, 19.3, 10. , 47.9, 19.1, 17.5, 19.8, 28. ,
       24.4, 11.1])

In [10]:
weights

array([-7.04099248, -5.2483589 , 25.09533916])

In [11]:
bias

13.761810393568448

In [13]:
X_train = np.column_stack((np.ones(x_train.shape[0]), x_train))
X_test = np.column_stack((np.ones(x_test.shape[0]), x_test))

coefficients = np.linalg.inv(X_train.T @ X_train) @ X_train.T @ y_train

y_pred_train = X_train @ coefficients

residuals = y_train - y_pred_train

n = len(y_train)
p = X_train.shape[1]

residual_variance = np.sum(residuals**2) / (n - p)

X_transpose_X_inv = np.linalg.inv(X_train.T @ X_train)
coeff_variance = residual_variance * X_transpose_X_inv

standard_errors = np.sqrt(np.diag(coeff_variance))

t_statistics = coefficients / standard_errors

p_values = [2 * (1 - stats.t.cdf(np.abs(t), n - p)) for t in t_statistics]

for i, (coef, t_stat, p_val) in enumerate(zip(coefficients, t_statistics, p_values)):
    print(f"Коэффициент {i}: {coef:.4f}, t-статистика = {t_stat:.4f}, p-value = {p_val:.4f}")

alpha = 0.05
significant = [p < alpha for p in p_values]

for i, sig in enumerate(significant):
    if sig:
        print(f"Коэффициент {i} значим (p < {alpha})")
    else:
        print(f"Коэффициент {i} не значим (p >= {alpha})")


Коэффициент 0: 10.0914, t-статистика = 3.4046, p-value = 0.0011
Коэффициент 1: -1.9374, t-статистика = -0.1328, p-value = 0.8948
Коэффициент 2: -4.2660, t-статистика = -0.2974, p-value = 0.7671
Коэффициент 3: 31.4913, t-статистика = 6.8164, p-value = 0.0000
Коэффициент 0 значим (p < 0.05)
Коэффициент 1 не значим (p >= 0.05)
Коэффициент 2 не значим (p >= 0.05)
Коэффициент 3 значим (p < 0.05)


In [15]:
y_pred = predict(x_train, weights, bias)
residuals = y_train - y_pred
residual_variance = np.sum(residuals**2) / (x_train.shape[0] - x_train.shape[1])
print("Остаточная дисперсия:")
print(residual_variance)

Остаточная дисперсия:
40.3691000830347


In [17]:
t_crit = stats.t.ppf(0.975, df=(x_train.shape[0] - x_train.shape[1]))
stderr = np.sqrt(residual_variance / (x_train.shape[0] - x_train.shape[1]))
conf_intervals = np.vstack((weights - t_crit * stderr, weights + t_crit * stderr)).T
print("Доверительные интервалы для коэффициентов модели:")
print(conf_intervals)

Доверительные интервалы для коэффициентов модели:
[[-8.60247056 -5.47951439]
 [-6.80983699 -3.68688082]
 [23.53386107 26.65681724]]


In [18]:
ss_total = np.sum((y_train - np.mean(y_train))**2)
ss_residual = np.sum(residuals**2)
r_squared = 1 - (ss_residual / ss_total)
print("Коэффициент детерминации R^2:")
print(r_squared)

Коэффициент детерминации R^2:
0.6039631547515167
